In [1]:
import numpy as np
import GetAllSubjectsSignalsDict as allSubjectsSignalsDict
import GetFeaturesAtEachEpochDict as featuresAtEachEpochDict
import dataFile as getData
from CalculateICC import CalculateICC

In [2]:
import cProfile, pstats, io



def profile(fnc):
    
    """A decorator that uses cProfile to profile a function"""
    
    def inner(*args, **kwargs):
        
        pr = cProfile.Profile()
        pr.enable()
        retval = fnc(*args, **kwargs)
        pr.disable()
        s = io.StringIO()
        sortby = 'cumulative'
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return retval

    return inner

In [6]:
EPOCHS_LIST = getData.getEpochList()
ELECTRODES_LIST = getData.getElectrodeList()

numEpochs = len(EPOCHS_LIST)
numElectrodes = len(ELECTRODES_LIST)

@profile
def getICCValues(featureName, referenceName):

    featureList = getData.getFeatureList(featureName)
    numFeatures = len(featureList)
    numFeatureValues = numEpochs*numFeatures*numElectrodes
    iccValuesArr = np.zeros(numFeatureValues)

    if (referenceName == 'averageReference'):
        signalsDict = allSubjectsSignalsDict.getAvgSignalsDict()
    elif(referenceName == 'czReference'):
        signalsDict = allSubjectsSignalsDict.getCzSignalsDict()
    else: 
        signalsDict = allSubjectsSignalsDict.getDefaultSignalsDict() 

    featuresDict = featuresAtEachEpochDict.getFeaturesAtEachEpochDict(signalsDict, featureName)

    for idx in range(0, numFeatureValues-1):
        icc = CalculateICC(featuresDict[idx])
        iccValuesArr[idx] = icc.calculateICC() 
    iccValuesArr = iccValuesArr.reshape(numEpochs, numFeatures, numElectrodes)
    return iccValuesArr

In [7]:
%time getICCValues('AbsolutePowerFeature', 'averageReference')

         11969586 function calls (11961476 primitive calls) in 284.141 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.051    0.051  284.141  284.141 /var/folders/w7/3zf2fgbs4vs99k494x3pmksh0000gn/T/ipykernel_6073/1357381946.py:7(getICCValues)
        1    0.292    0.292  283.363  283.363 /Users/akarshijain/EEGFeatureReliability/GetFeaturesAtEachEpochDict.py:34(getFeaturesAtEachEpochDict)
       10    0.000    0.000  253.138   25.314 /Users/akarshijain/EEGFeatureReliability/GetFilterSignals.py:3(getFilterSignals)
       60    0.176    0.003  253.138    4.219 /Users/akarshijain/EEGFeatureReliability/signalFilters.py:175(filterSignalDict)
     1140    8.115    0.007  252.822    0.222 /Users/akarshijain/opt/anaconda3/lib/python3.9/site-packages/scipy/signal/signaltools.py:3885(filtfilt)
     1140   28.296    0.025  244.685    0.215 /Users/akarshijain/opt/anaconda3/lib/python3.9/site-packages/scipy/signal/signa

array([[[0.06421318, 0.11342621, 0.48685751, 0.57633292, 0.14050352,
         0.2398204 , 0.05628022, 0.09968661, 0.46840729, 0.57924854,
         0.06584584, 0.1179539 , 0.18076904, 0.2755532 , 0.51354399,
         0.59759066, 0.26075828, 0.38747188, 0.1716734 ],
        [0.2621769 , 0.65100446, 0.74498596, 0.15340947, 0.25109225,
         0.63017936, 0.72594282, 0.5252597 , 0.62097261, 0.44388832,
         0.58001036, 0.5060673 , 0.63678615, 0.02670517, 0.04734924,
         0.4134294 , 0.53896728, 0.00701573, 0.01281422],
        [0.53408203, 0.62258712, 0.05764952, 0.10657959, 0.00217215,
         0.00367245, 0.06697365, 0.11599611, 0.01574169, 0.03051887,
         0.6066265 , 0.72461135, 0.60440112, 0.72577385, 0.52481488,
         0.6344807 , 0.22271703, 0.32625149, 0.05239835],
        [0.08330309, 0.51029517, 0.63287811, 0.62531758, 0.73268671,
         0.59354629, 0.70839567, 0.495522  , 0.6010364 , 0.23474411,
         0.35352057, 0.26449003, 0.31836778, 0.41017449, 0.53695719